In [9]:
import pandas as pd
import yfinance as yf

# Specify the stock you want to analyze (e.g., Medco)
# AAPL
# MSFT
# GOOG
ticker = yf.Ticker('MSFT')

In [10]:
ticker.balance_sheet.columns

DatetimeIndex(['2024-06-30', '2023-06-30', '2022-06-30', '2021-06-30'], dtype='datetime64[ns]', freq=None)

In [11]:
balance_sheet = ticker.balancesheet
income_statement = ticker.financials
cash_flow = ticker.cashflow

In [12]:
base_year = balance_sheet.columns[-1]

In [13]:
base_balance_sheet = {
    'total_assets': {
        'current_assets': {
            'cash_and_equivalents': balance_sheet[base_year]['Cash Cash Equivalents And Short Term Investments'],
            'receivables': balance_sheet[base_year]['Receivables'],
            'Inventory': balance_sheet[base_year]['Inventory'],
            'hedging_assets_current': balance_sheet[base_year]['Hedging Assets Current'],
            'other_current_assets': balance_sheet[base_year]['Other Current Assets']
        },
        'non_current_assets': {
            'net_ppe': balance_sheet[base_year]['Net PPE'],
            'investments': balance_sheet[base_year]['Investments And Advances'],
            'goodwill': balance_sheet[base_year]['Goodwill And Other Intangible Assets'],
            'other_non_current_assets': balance_sheet[base_year]['Other Non Current Assets']
        }
    },
    'total_liabilities': {
        'current_liabilities': {
            'payables_and_accrued_expenses': balance_sheet[base_year]['Payables And Accrued Expenses'],
            'pension_and_other_postretirement_benefits': balance_sheet[base_year]['Pensionand Other Post Retirement Benefit Plans Current'],
            'current_debt': balance_sheet[base_year]['Current Debt And Capital Lease Obligation'],
            'current_deferred_liabilities': balance_sheet[base_year]['Current Deferred Liabilities'],
            'other_current_liabilities': balance_sheet[base_year]['Other Current Liabilities']
        },
        'non_current_liabilities': {
            'long_term_debt': balance_sheet[base_year]['Long Term Debt And Capital Lease Obligation'],
            'non_current_deferred_liabilities': balance_sheet[base_year]['Non Current Deferred Liabilities'],
            'tradeand_other_payables_non_current': balance_sheet[base_year]['Tradeand Other Payables Non Current'],
            'other_non_current_liabilities': balance_sheet[base_year]['Other Non Current Liabilities']
        }
    },
    'total_equity': {
        'capital_stock': balance_sheet[base_year]['Capital Stock'],
        'retained_earnings': balance_sheet[base_year]['Retained Earnings'],
        'gains_losses_not_affecting_retained_earning': balance_sheet[base_year]['Gains Losses Not Affecting Retained Earnings'],
    }
}

# write a recursive function that pass in a dictionary and return the sum of all values in the dictionary
def sum_dict_values(d):
    total = 0
    for key, value in d.items():
        if isinstance(value, dict):
            total += sum_dict_values(value)
        else:
            total += value
    return total
assert sum_dict_values(base_balance_sheet['total_assets']) == balance_sheet[base_year]['Total Assets'], "Total assets do not match the sum of current and non-current assets."
assert sum_dict_values(base_balance_sheet['total_liabilities']) == balance_sheet[base_year]['Total Liabilities Net Minority Interest'], "Total liabilities do not match the sum of current and non-current liabilities."
assert sum_dict_values(base_balance_sheet['total_equity']) == balance_sheet[base_year]['Total Equity Gross Minority Interest'], "Total equity does not match the sum of capital stock, retained earnings, and gains/losses not affecting retained earnings."
assert sum_dict_values(base_balance_sheet['total_assets']) - sum_dict_values(base_balance_sheet['total_liabilities']) - sum_dict_values(base_balance_sheet['total_equity']) == 0, "The balance sheet equation does not hold: Total Assets = Total Liabilities + Total Equity."
assert balance_sheet[base_year]['Total Assets'] - balance_sheet[base_year]['Total Liabilities Net Minority Interest'] - balance_sheet[base_year]['Total Equity Gross Minority Interest'] == 0, "The real balance sheet equation does not hold: Total Assets = Total Liabilities + Total Equity."
assert sum_dict_values(base_balance_sheet) == (balance_sheet[base_year]['Total Assets'] + balance_sheet[base_year]['Total Liabilities Net Minority Interest'] + balance_sheet[base_year]['Total Equity Gross Minority Interest']), "Total balance sheet does not match the actual sum of total assets, total liabilities, and total equity."

In [14]:
base_income_statement = {
    'total_revenue': income_statement[base_year]['Total Revenue'],
    'cost_of_revenue': income_statement[base_year]['Cost Of Revenue'],
    'gross_profit': income_statement[base_year]['Gross Profit'],
    'operating_expenses': income_statement[base_year]['Operating Expense'],
    'operating_income': income_statement[base_year]['Operating Income'],
    'net_non_operating_interest_income_expense': income_statement[base_year]['Net Non Operating Interest Income Expense'],
    'other_income_expense': income_statement[base_year]['Other Income Expense'],
    'pretax_income': income_statement[base_year]['Pretax Income'],
    'tax_provision': income_statement[base_year]['Tax Provision'],
    'net_income_common_stockholders': income_statement[base_year]['Net Income Common Stockholders'],
}

assert base_income_statement['total_revenue'] - base_income_statement['cost_of_revenue'] == base_income_statement['gross_profit'], "Gross profit does not match total revenue minus cost of revenue."
assert base_income_statement['gross_profit'] - base_income_statement['operating_expenses'] == base_income_statement['operating_income'], "Operating income does not match gross profit minus operating expenses."
assert base_income_statement['operating_income'] + base_income_statement['net_non_operating_interest_income_expense'] + base_income_statement['other_income_expense'] == base_income_statement['pretax_income'], "Pretax income does not match operating income minus net non-operating interest income/expense and other income/expense."
assert base_income_statement['pretax_income'] - base_income_statement['tax_provision'] == base_income_statement['net_income_common_stockholders'], "Net income does not match pretax income minus tax provision."

In [15]:
base_cash_flow = {
    'capital_expenditure': cash_flow[base_year]['Capital Expenditure'],
    'depreciation_amortization_depletion': cash_flow[base_year]['Depreciation Amortization Depletion'],
    'cash_dividends_paid': cash_flow[base_year]['Cash Dividends Paid'],
}

In [16]:
meta_attributes = {
    'sales_growth_rate': 1.05,
    'operating_margin': base_income_statement['operating_income'] / base_income_statement['total_revenue'],
    'capex_as_percentage_of_sales': base_cash_flow['capital_expenditure'] / base_income_statement['total_revenue'],
    'depreciation_amortization_depletion_as_percentage_of_net_ppe': base_cash_flow['depreciation_amortization_depletion'] / base_balance_sheet['total_assets']['non_current_assets']['net_ppe'],
    'interest_rate_on_debt': base_income_statement['net_non_operating_interest_income_expense'] / (base_balance_sheet['total_liabilities']['current_liabilities']['current_debt'] + base_balance_sheet['total_liabilities']['non_current_liabilities']['long_term_debt']),
    'tax_rate': base_income_statement['tax_provision'] / base_income_statement['pretax_income'],
    'dividend_payout_ratio': base_cash_flow['cash_dividends_paid'] / base_income_statement['net_income_common_stockholders'],
    'minimum_cash_required': base_balance_sheet['total_assets']['current_assets']['cash_and_equivalents'],
}

In [17]:
# --- "No Change" Sanity Check Assumptions ---

meta_attributes = {
    # 1. No growth in sales.
    'sales_growth_rate': 1.0,  # representing 0% growth.

    # 2. Margins and historical ratios remain the same. These are correctly defined.
    'operating_margin': base_income_statement['operating_income'] / base_income_statement['total_revenue'],
    'capex_as_percentage_of_sales': base_cash_flow['capital_expenditure'] / base_income_statement['total_revenue'],
    'depreciation_amortization_depletion_as_percentage_of_net_ppe': base_cash_flow['depreciation_amortization_depletion'] / base_balance_sheet['total_assets']['non_current_assets']['net_ppe'], # Corrected to be % of Net PPE
    'interest_rate_on_debt': base_income_statement['net_non_operating_interest_income_expense'] / (base_balance_sheet['total_liabilities']['current_liabilities']['current_debt'] + base_balance_sheet['total_liabilities']['non_current_liabilities']['long_term_debt']),
    'tax_rate': base_income_statement['tax_provision'] / base_income_statement['pretax_income'],
    'dividend_payout_ratio': 1.0,

    # 3. CRITICAL: For a true "no change", Capex must equal Depreciation.
    #    We will enforce this in the model logic itself rather than here.
    #    The logic will be: Forecast_Depreciation = Net_PPE(t) * depreciation_rate
    #                       Forecast_Capex = Forecast_Depreciation
    #    This ensures the Net PPE base does not grow or shrink.

    # 4. Minimum cash required is the same as last year's ending cash.
    'minimum_cash_required': base_balance_sheet['total_assets']['current_assets']['cash_and_equivalents'],
}


In [18]:
# --- Start with Base Year (t) Data ---
# All your 'base_balance_sheet', 'base_income_statement', etc. data is here.

# --- Use "No Change" Assumptions ---
# The 'meta_attributes' dictionary we defined for the sanity check.

# --- Create a dictionary to hold the forecast results for Year t+1 ---
forecast_t_plus_1 = {}

# --- Extract key base year numbers for clarity ---
sales_t = base_income_statement['total_revenue']
net_ppe_t = base_balance_sheet['total_assets']['non_current_assets']['net_ppe']
total_debt_t = base_balance_sheet['total_liabilities']['current_liabilities']['current_debt'] + base_balance_sheet['total_liabilities']['non_current_liabilities']['long_term_debt']
cash_t = base_balance_sheet['total_assets']['current_assets']['cash_and_equivalents']
retained_earnings_t = base_balance_sheet['total_equity']['retained_earnings']
common_stock_t = base_balance_sheet['total_equity']['capital_stock']


In [19]:
# --- Top-to-bottom calculation ---

forecast_t_plus_1['sales'] = sales_t * meta_attributes['sales_growth_rate']  # sales_growth_rate = 1.0

forecast_t_plus_1['operating_income'] = forecast_t_plus_1['sales'] * meta_attributes['operating_margin']

# Calculate Depreciation for t+1 first, as it's needed for EBIT and for the Capex assumption
forecast_t_plus_1['depreciation'] = net_ppe_t * meta_attributes['depreciation_amortization_depletion_as_percentage_of_net_ppe']

# Note: In a full model, EBIT would be Sales - COGS - OpEx. Here, 'operating_income' is our proxy for EBIT.
# For simplicity, we assume 'operating_income' is post-depreciation.
# If 'operating_income' was pre-depreciation (like EBITDA), the line would be:
# forecast_t_plus_1['ebit'] = forecast_t_plus_1['operating_income'] - forecast_t_plus_1['depreciation']
# Let's assume your 'operating_income' is already EBIT for this exercise.
forecast_t_plus_1['ebit'] = forecast_t_plus_1['operating_income']

forecast_t_plus_1['interest_expense'] = total_debt_t * meta_attributes['interest_rate_on_debt']
# <-- KEY "NO CIRCULARITY" STEP: Interest is based on KNOWN debt from Year t.

forecast_t_plus_1['pretax_income'] = forecast_t_plus_1['ebit'] - forecast_t_plus_1['interest_expense']

forecast_t_plus_1['tax_provision'] = forecast_t_plus_1['pretax_income'] * meta_attributes['tax_rate']

forecast_t_plus_1['net_income'] = forecast_t_plus_1['pretax_income'] - forecast_t_plus_1['tax_provision']

In [20]:
# --- Calculate Cash Flows from Operations, Investing, and Financing ---

# Cash Flow from Operations (CFO) - Simple Version
# Start with Net Income and add back non-cash Depreciation
cfo_t_plus_1 = forecast_t_plus_1['net_income'] + forecast_t_plus_1['depreciation']

# Cash Flow from Investing (CFI)
# *** ENFORCE THE "NO CHANGE" RULE: Capex = Depreciation ***
forecast_t_plus_1['capex'] = forecast_t_plus_1['depreciation']
cfi_t_plus_1 = -forecast_t_plus_1['capex']

# Scheduled Cash Flow from Financing (CFF)
forecast_t_plus_1['dividends_paid'] = forecast_t_plus_1['net_income'] * meta_attributes['dividend_payout_ratio']
scheduled_cff_t_plus_1 = -forecast_t_plus_1['dividends_paid']

# --- Calculate the company's cash position BEFORE any new borrowing or investing ---
net_cash_flow_before_new_financing = cfo_t_plus_1 + cfi_t_plus_1 + scheduled_cff_t_plus_1

provisional_cash_balance = cash_t + net_cash_flow_before_new_financing

# --- The "No-Plug" Decision Logic ---
cash_surplus_or_deficit = provisional_cash_balance - meta_attributes['minimum_cash_required']

if cash_surplus_or_deficit < 0:
    forecast_t_plus_1['new_debt_needed'] = -cash_surplus_or_deficit
    forecast_t_plus_1['new_st_investment'] = 0
else:
    forecast_t_plus_1['new_debt_needed'] = 0
    forecast_t_plus_1['new_st_investment'] = cash_surplus_or_deficit

# FOR THE "NO CHANGE" TEST, WE EXPECT 'new_debt_needed' AND 'new_st_investment' TO BE ZERO!

In [21]:
# --- Create a new, structured dictionary for the BS t+1 ---
# It will have the same structure as your 'base_balance_sheet'
forecast_bs_t_plus_1 = {
    'total_assets': {'current_assets': {}, 'non_current_assets': {}},
    'total_liabilities': {'current_liabilities': {}, 'non_current_liabilities': {}},
    'total_equity': {}
}

#=====================================================================
# ASSETS Side: Update the items that change, carry over the rest
#=====================================================================

# --- Current Assets ---
forecast_bs_t_plus_1['total_assets']['current_assets']['cash_and_equivalents'] = cash_t + net_cash_flow_before_new_financing + forecast_t_plus_1['new_debt_needed'] - forecast_t_plus_1['new_st_investment']
forecast_bs_t_plus_1['total_assets']['current_assets']['receivables'] = base_balance_sheet['total_assets']['current_assets']['receivables']
forecast_bs_t_plus_1['total_assets']['current_assets']['Inventory'] = base_balance_sheet['total_assets']['current_assets']['Inventory']
forecast_bs_t_plus_1['total_assets']['current_assets']['hedging_assets_current'] = base_balance_sheet['total_assets']['current_assets']['hedging_assets_current']
forecast_bs_t_plus_1['total_assets']['current_assets']['other_current_assets'] = base_balance_sheet['total_assets']['current_assets']['other_current_assets']

# --- Non-Current Assets ---
net_ppe_t = base_balance_sheet['total_assets']['non_current_assets']['net_ppe']
forecast_bs_t_plus_1['total_assets']['non_current_assets']['net_ppe'] = net_ppe_t - forecast_t_plus_1['depreciation'] + forecast_t_plus_1['capex']

investments_t = base_balance_sheet['total_assets']['non_current_assets']['investments']
forecast_bs_t_plus_1['total_assets']['non_current_assets']['investments'] = investments_t + forecast_t_plus_1['new_st_investment']

forecast_bs_t_plus_1['total_assets']['non_current_assets']['goodwill'] = base_balance_sheet['total_assets']['non_current_assets']['goodwill']
forecast_bs_t_plus_1['total_assets']['non_current_assets']['other_non_current_assets'] = base_balance_sheet['total_assets']['non_current_assets']['other_non_current_assets']

#=====================================================================
# LIABILITIES & EQUITY Side: Update the items that change, carry over the rest
#=====================================================================

# --- Current Liabilities ---
forecast_bs_t_plus_1['total_liabilities']['current_liabilities']['payables_and_accrued_expenses'] = base_balance_sheet['total_liabilities']['current_liabilities']['payables_and_accrued_expenses']
forecast_bs_t_plus_1['total_liabilities']['current_liabilities']['pension_and_other_postretirement_benefits'] = base_balance_sheet['total_liabilities']['current_liabilities']['pension_and_other_postretirement_benefits']

# THIS IS THE CORRECTED LINE: using your 'current_debt' key
forecast_bs_t_plus_1['total_liabilities']['current_liabilities']['current_debt'] = base_balance_sheet['total_liabilities']['current_liabilities']['current_debt']

forecast_bs_t_plus_1['total_liabilities']['current_liabilities']['current_deferred_liabilities'] = base_balance_sheet['total_liabilities']['current_liabilities']['current_deferred_liabilities']
forecast_bs_t_plus_1['total_liabilities']['current_liabilities']['other_current_liabilities'] = base_balance_sheet['total_liabilities']['current_liabilities']['other_current_liabilities']

# --- Non-Current Liabilities ---
# THIS IS THE CORRECTED LINE: using your 'long_term_debt' key
long_term_debt_t = base_balance_sheet['total_liabilities']['non_current_liabilities']['long_term_debt']
forecast_bs_t_plus_1['total_liabilities']['non_current_liabilities']['long_term_debt'] = long_term_debt_t + forecast_t_plus_1['new_debt_needed']

forecast_bs_t_plus_1['total_liabilities']['non_current_liabilities']['non_current_deferred_liabilities'] = base_balance_sheet['total_liabilities']['non_current_liabilities']['non_current_deferred_liabilities']
forecast_bs_t_plus_1['total_liabilities']['non_current_liabilities']['tradeand_other_payables_non_current'] = base_balance_sheet['total_liabilities']['non_current_liabilities']['tradeand_other_payables_non_current']
forecast_bs_t_plus_1['total_liabilities']['non_current_liabilities']['other_non_current_liabilities'] = base_balance_sheet['total_liabilities']['non_current_liabilities']['other_non_current_liabilities']

# --- Total Equity ---
forecast_bs_t_plus_1['total_equity']['capital_stock'] = base_balance_sheet['total_equity']['capital_stock']

retained_earnings_t = base_balance_sheet['total_equity']['retained_earnings']
forecast_bs_t_plus_1['total_equity']['retained_earnings'] = retained_earnings_t + forecast_t_plus_1['net_income'] - forecast_t_plus_1['dividends_paid']

forecast_bs_t_plus_1['total_equity']['gains_losses_not_affecting_retained_earning'] = base_balance_sheet['total_equity']['gains_losses_not_affecting_retained_earning']

In [22]:
def sum_assets(bs):
    total = 0
    for category in bs['total_assets'].values():
        for item_value in category.values():
            total += item_value
    return total

def sum_liabilities_and_equity(bs):
    total = 0
    for category in bs['total_liabilities'].values():
        for item_value in category.values():
            total += item_value
    for item_value in bs['total_equity'].values():
        total += item_value
    return total

# --- The Complete and Rigorous Moment of Truth ---
total_assets_t_plus_1 = sum_assets(forecast_bs_t_plus_1)
total_liabilities_and_equity_t_plus_1 = sum_liabilities_and_equity(forecast_bs_t_plus_1)

balance_check = total_assets_t_plus_1 - total_liabilities_and_equity_t_plus_1

print(f"Total Assets (t+1): {total_assets_t_plus_1}")
print(f"Total Liabilities & Equity (t+1): {total_liabilities_and_equity_t_plus_1}")
print(f"Final Balance Check: {balance_check}")

# --- You should assert this value is extremely close to zero ---
assert abs(balance_check) < 1e-6, "BALANCE SHEET DOES NOT BALANCE!"

print("Sanity Check Passed: The complete, detailed model balances correctly.")

Total Assets (t+1): 333779000000.0
Total Liabilities & Equity (t+1): 333779000000.0
Final Balance Check: 0.0
Sanity Check Passed: The complete, detailed model balances correctly.


In [23]:
sum_dict_values(base_balance_sheet["total_assets"])

333779000000.0

In [24]:
# recursively print the base_balance_sheet dictionary keys and values
def print_balance_sheet(d, indent=0):
    for key, value in d.items():
        print(' ' * indent + str(key) + ':')
        if isinstance(value, dict):
            print_balance_sheet(value, indent + 2)
        else:
            pass
            # print(' ' * (indent + 2) + str(value))
# print("Base Balance Sheet:")
print_balance_sheet(base_balance_sheet)

total_assets:
  current_assets:
    cash_and_equivalents:
    receivables:
    Inventory:
    hedging_assets_current:
    other_current_assets:
  non_current_assets:
    net_ppe:
    investments:
    goodwill:
    other_non_current_assets:
total_liabilities:
  current_liabilities:
    payables_and_accrued_expenses:
    pension_and_other_postretirement_benefits:
    current_debt:
    current_deferred_liabilities:
    other_current_liabilities:
  non_current_liabilities:
    long_term_debt:
    non_current_deferred_liabilities:
    tradeand_other_payables_non_current:
    other_non_current_liabilities:
total_equity:
  capital_stock:
  retained_earnings:
  gains_losses_not_affecting_retained_earning:
